In [ ]:
import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score 

import pandas as pd

In [ ]:
df = pd.read_csv('admissions_data.csv')
df.columns = df.columns.str.strip()
features = df
features = features.drop('Serial No.', axis=1)
features = features.drop('Chance of Admit', axis=1)
print(features)

In [ ]:
lables = df['Chance of Admit']
print(lables)

In [ ]:
#break features and lables into train and test sets
features_train, features_test, lables_train, lables_test = train_test_split(features, lables, test_size=0.2, random_state=42)
print(f"features_train \n {features_train}")
print(f"features_test \n {features_test}")
print(f"lables_test \n {lables_test}")
print(f"lables_train \n {lables_train}")

In [ ]:
#scale data so they follow the same range
features_scaler = StandardScaler()
features_train_scaled = features_scaler.fit_transform(features_train)
features_test_scaled = features_scaler.transform(features_test)
lables_scaler = StandardScaler()
lables_train_scaled = lables_scaler.fit_transform(lables_train.values.reshape(-1,1)).flatten()
lables_test_scaled = lables_scaler.transform(lables_test.values.reshape(-1,1)).flatten()

In [ ]:
def make_model(feat_train, labels_train):
    model = Sequential

    

    model.fit(features_train_scaled, lables_train_scaled)

    return model 

In [ ]:
my_model = Sequential()

my_model.add(keras.Input(shape=(16,)))
my_model.add(Dense(8, activation='relu'))
my_model.add(Dense(6))



my_model.summary()